# DETR Object Detection for localization and classification of thoracic abnormalities 
Authors: Haris Mpournas & Elena Stamatelou


!sudo apt-get update
!apt-cache search libgl
!sudo apt-get install libgl1-mesa-glx -y

#%load /home/ttban9527/.local/lib/python3.10/site-packages/albumentations/core/bbox_utils.py
#%load /home/ttban9527/.local/lib/python3.10/site-packages/map_boxes/__init__.py

## 1. Select mode
mode='train' for the training, mode='predict' for the predictions
uncomment the one you want to enable and comment the one you want to enable 

In [1]:
mode = 'train'
#mode = 'predict'

#### For **mode = 'train**',
1) Add data --> Competitions Data --> Search for "VinBigData Chest X-ray Abnormalities Detection" </br>
2) Add data --> Datasets --> Search for "vinbigdata-chest-xray-original-png"</br>
3) Enable the GPU in the Settings --> Accelarator --> GPU</br>
The output of the mode is "detr_model.pth"</br>

#### For **mode = 'predict'**, 
1) Go to the outputs of the previous mode (train mode "detr_model.pth"), select "Add new version" and keep the created URL</br>
2) Go back to Kaggle's notebook --> Add data --> Datasets --> Search by URL with the saved URL from the last step</br>
3) Εnable the CPU in the Settings --> Accelarator --> CPU</br>
 

## 2. Import libraries

In [2]:
# clone github repo of detr
#!git clone https://github.com/facebookresearch/detr.git   

# general libraries
import os
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
import random
from tqdm.autonotebook import tqdm
import re
import pydicom
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
from PIL import Image

# torch.
import torch
import torch.nn as nn

import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler

from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset

# sklearn
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection

# CV
import cv2

# DETR FUCNTIONS FOR LOSS
import sys
sys.path.append('./detr/')

from detr.models.matcher import HungarianMatcher
from detr.models.detr import SetCriterion

# albumenatations
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2

# Glob
from glob import glob

# ensembling 
#!pip install ensemble_boxes
from tqdm.autonotebook import tqdm
from ensemble_boxes import *

# mAP
#!pip install map_boxes
from map_boxes import mean_average_precision_for_boxes

import ast

/tmp/ipykernel_7274/2505089380.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
def seed_everything(seed):
    # Set Python random seed
    random.seed(seed)
    
    # Set NumPy random seed
    np.random.seed(seed)
    
    # Set PyTorch random seed for CPU and GPU
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    # Set PyTorch deterministic operations for cudnn backend
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
from detr.util.misc import nested_tensor_from_tensor_list
class NoOpAttacker():
    def attack(self, image, label, model):
        return image

class PGDAttacker():
    def __init__(self, num_iter, epsilon, step_size, device):
        self.num_iter = num_iter
        self.epsilon = epsilon / 255
        self.step_size = step_size / 255
        self.device = device

    def attack(self, image_clean, target, model):     
        # 設定上下界
        seed_everything(seed)
        lower_bound = torch.clamp(image_clean - self.epsilon, min=-1., max=1.)
        upper_bound = torch.clamp(image_clean + self.epsilon, min=-1., max=1.)

        # 隨機初始化
        init_start = torch.empty_like(image_clean).uniform_(-self.epsilon, self.epsilon)
        adv = (image_clean + init_start).to(self.device)

        # 設定損失函數
        matcher = HungarianMatcher()
        weight_dict = {'loss_ce': 1, 'loss_bbox': 1, 'loss_giou': 1}
        losses = ['labels', 'boxes', 'cardinality']
        criterion = SetCriterion(num_classes, matcher, weight_dict, eos_coef = null_class_coef, losses=losses)
        criterion = criterion.to(self.device)
        
        for _ in range(self.num_iter):
            adv.requires_grad = True
            
            #adv_detached = adv.clone().detach()
            samples = nested_tensor_from_tensor_list([adv])
            
            output = model(samples)
            
            loss_dict = criterion(output, [target])
            weight_dict = criterion.weight_dict
            losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
            
            g = torch.autograd.grad(losses, adv, retain_graph=False, create_graph=False, allow_unused=True)[0]
            
            adv = adv + torch.sign(g) * self.step_size
            adv = torch.clamp(adv, min=lower_bound, max=upper_bound).clone().detach()

        return adv

## Def normalize_bboxes、denormalize_bboxes

In [5]:
# thoracic abnormalities (classes)
CLASSES = [
    'Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation',
    'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 
    'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis', 'No Finding'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

## 3. Preprocessing image metadata

In [6]:
def read_images():
    # read the images with size 512x512 
    # add the dataset in the data section if it is not added yet
    train_df = pd.read_csv("/home/ttban9527/vinbigdata/train.csv")
    train_df.fillna(0, inplace=True)
    return train_df


In [7]:
def clean_data(df):
    df_class_14 = df[df['class_id'] == 14]
    df_class_14_unique = df_class_14.drop_duplicates(subset=['image_id'], keep='first')

    df_other_classes = df[df['class_id'] != 14]

    df_combined = pd.DataFrame()

    for image_id in df_other_classes['image_id'].unique():
        df_image = df_other_classes[df_other_classes['image_id'] == image_id]
    
        df_radiologists = pd.DataFrame()
        
        for class_id, group in df_image.groupby('class_id'):
            if len(group) > 1:
                overlapped = np.zeros(len(group), dtype=bool)
                for i in range(len(group)):
                    if overlapped[i]:
                        continue
                    for j in range(i+1, len(group)):
                        if (group.iloc[i]['x_min'] < group.iloc[j]['x_max'] and
                            group.iloc[i]['x_max'] > group.iloc[j]['x_min'] and
                            group.iloc[i]['y_min'] < group.iloc[j]['y_max'] and
                            group.iloc[i]['y_max'] > group.iloc[j]['y_min']):
                            overlapped[j] = True
            
                for i in range(len(group)):
                    if not overlapped[i]:
                        df_radiologists = pd.concat([df_radiologists, group.iloc[[i]]])
            else:
                df_radiologists = pd.concat([df_radiologists, group])
    
        df_combined = pd.concat([df_combined, df_radiologists])

    df_combined = pd.concat([df_combined, df_class_14_unique])

    df_combined = df_combined.sample(frac=1, random_state=42)

    df_combined.reset_index(drop=True, inplace=True)
    
    return df_combined

### 3.1 Scale images

In [8]:
def scale_images(train_df):
    # scale the coordinates of the bounding boxes from their initial values to fit the 512x512 images
    # set to the images with no object (class 14), bounding box with coordinates [xmin=0 ymin=0 xmax=1 ymax=1]
    train_df.loc[train_df["class_id"] == 14, ['x_max', 'y_max']] = 1.0
    train_df.loc[train_df["class_id"] == 14, ['x_min', 'y_min']] = 0

    # scale the input image coordinates to fit 512x512 image
    IMG_SIZE = 512
    train_df['xmin'] = (train_df['x_min']/train_df['width'])*IMG_SIZE
    train_df['ymin'] = (train_df['y_min']/train_df['height'])*IMG_SIZE
    train_df['xmax'] = (train_df['x_max']/train_df['width'])*IMG_SIZE
    train_df['ymax'] = (train_df['y_max']/train_df['height'])*IMG_SIZE

    # set to the images with no object (class 14), bounding box with coordinates [xmin=0 ymin=0 xmax=1 ymax=1]
    train_df.loc[train_df["class_id"] == 14, ['xmax', 'ymax']] = 1.0
    train_df.loc[train_df["class_id"] == 14, ['xmin', 'ymin']] = 0
    return train_df

### 3.2 Define folds

In [9]:
def define_folds(train_df):
    unique_images = train_df["image_id"].unique()
    df_split = pd.DataFrame(unique_images, columns = ['unique_images']) 

    # create one column with the number of fold (for the k-fold cross validation)
    df_split["kfold"] = -1
    df_split = df_split.sample(frac=1).reset_index(drop=True)
    y = df_split.unique_images.values
    kf = model_selection.GroupKFold(n_splits=5)
    for f, (t_, v_) in enumerate(kf.split(X=df_split, y=y, groups=df_split.unique_images.values)):
        df_split.loc[v_, "kfold"] = f

    # annotated boxes from same "image id" (image) should be in the same fold [during training each image with its boxes is as one input]
    train_df["kfold"] = -1
    for ind in train_df.index: 
         train_df["kfold"][ind] = df_split.loc[ df_split["unique_images"] ==  train_df["image_id"][ind]]["kfold"]

    train_df.set_index('image_id', inplace=True)
    return train_df

### 3.3 Weight boxes fusion 

In [10]:
def boxes_fusion(df):
    # apply weighted boxes fusion for ensemling overlapping annotated boxes
    # Default WBF config 
    iou_thr = 0.75
    skip_box_thr = 0.0001
    sigma = 0.1
    results = []
    image_ids = df.index.unique()
   
    for image_id in tqdm(image_ids, total=len(image_ids)):
        # All annotations for the current image.
        data = df[df.index == image_id]
        kfold = data['kfold'].unique()[0]
        data = data.reset_index(drop=True)
        
        # WBF expects the coordinates in 0-1 range.
        max_value = data.iloc[:, 4:].values.max()
        data.loc[:, ["xmin", "ymin", "xmax", "ymax"]] = data.iloc[:, 4:] / max_value
        #print("data",data)
        if data.class_id.unique()[0] !=14:
            annotations = {}
            weights = []
            # Loop through all of the annotations
            for idx, row in data.iterrows():
                rad_id = row["rad_id"]
                if rad_id not in annotations:
                    annotations[rad_id] = {
                        "boxes_list": [],
                        "scores_list": [],
                        "labels_list": [],
                    }
                    # We consider all of the radiologists as equal.
                    weights.append(1.0)
                annotations[rad_id]["boxes_list"].append([row["xmin"], row["ymin"], row["xmax"], row["ymax"]])
                annotations[rad_id]["scores_list"].append(1.0)
                annotations[rad_id]["labels_list"].append(row["class_id"])

            boxes_list = []
            scores_list = []
            labels_list = []

            for annotator in annotations.keys():
                boxes_list.append(annotations[annotator]["boxes_list"])
                scores_list.append(annotations[annotator]["scores_list"])
                labels_list.append(annotations[annotator]["labels_list"])

            # Calculate WBF
            boxes, scores, labels = weighted_boxes_fusion(boxes_list,
                scores_list,
                labels_list,
                weights=weights,
                iou_thr=iou_thr,
                skip_box_thr=skip_box_thr
            )
            for idx, box in enumerate(boxes):
                results.append({
                    "image_id": image_id,
                    "class_id": int(labels[idx]),
                    "rad_id": "wbf",
                    "xmin": box[0]* max_value,
                    "ymin": box[1]* max_value,
                    "xmax": box[2]* max_value,
                    "ymax": box[3]* max_value,
                    "kfold":kfold,
                })
        # if class is nothing then have it once (instead of 3 times in the same image)
        if data.class_id.unique()[0] ==14:
            for idx, box in enumerate([0]):
                results.append({
                    "image_id": image_id,
                    "class_id": data.class_id[0],
                    "rad_id": "wbf",
                    "xmin": 0,
                    "ymin": 0,
                    "xmax": 1,
                    "ymax": 1,
                    "kfold":kfold,
                })
            
    results = pd.DataFrame(results)
    return results

### 3.4 Pascal to coco

In [11]:
def pascal_to_coco(train_df):
    # Good exlanation of coco, pascal etc 
    # https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    
    train_df['coco_x'] = train_df['xmin'] + (train_df['xmax'] - train_df['xmin'] )/2
    train_df['coco_y'] = train_df['ymin'] + (train_df['ymax'] - train_df['ymin'] )/2
    train_df['coco_w'] = train_df['xmax'] - train_df['xmin'] 
    train_df['coco_h'] = train_df['ymax'] - train_df['ymin'] 

    train_df.loc[train_df['class_id'] == 14, 'coco_x'] = 1
    train_df.loc[train_df['class_id'] == 14, 'coco_y'] = 1
    train_df.loc[train_df['class_id'] == 14, 'coco_w'] = 0.5
    train_df.loc[train_df['class_id'] == 14, 'coco_h'] = 0.5
    
    return train_df

### 3.5 Main preprocessing function

In [12]:
def preprocessing():
    train_df = read_images()
    train_df = scale_images(train_df)
    train_df = define_folds(train_df)
    train_df = boxes_fusion(train_df)
    train_df['image_index'] = train_df['image_id']
    train_df.set_index('image_index', inplace=True)
    train_df = pascal_to_coco(train_df)
    return train_df

## 4. Creating Image Dataset class


In [13]:
def get_train_transforms():
    # image augmentations for the training set
    return A.Compose([A.ToGray(p=0.01),
                      #A.Cutout(num_holes=10, max_h_size=32, max_w_size=32, fill_value=0, p=0.5),
                      ToTensorV2(p=1.0)],
                      p=1.0,
                      bbox_params=A.BboxParams(format='coco',min_area=0, min_visibility=0,label_fields=['labels'])
                      )

def get_valid_transforms():
    # image augmentations for the validation set
    return A.Compose([ToTensorV2(p=1.0)], 
                      p=1.0, 
                      bbox_params=A.BboxParams(format='coco',min_area=0, min_visibility=0,label_fields=['labels'])
                      )

In [14]:
DIR_TRAIN_PNG = "/home/ttban9527/vinbigdata/train"

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

class VinDataset(Dataset):
    def __init__(self,image_ids,dataframe,transforms=None):
        self.image_ids = image_ids
        self.df = dataframe
        self.transforms = transforms
        
    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self,index):
        image_id = self.image_ids[index]
        records = self.df.loc[image_id]
        labels = records['class_id']
        
        image = cv2.imread(f'{DIR_TRAIN_PNG}/{image_id}.png', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        
        # DETR takes in data in coco format    
        boxes = records[['coco_x', 'coco_y', 'coco_w', 'coco_h']].values
     
        # AS pointed out by PRVI It works better if the main class is labelled as zero
        labels =  np.array(labels)
    
        if boxes.ndim == 1 : 
            boxes = np.expand_dims(boxes, axis=0)
            labels = np.expand_dims(labels, axis=0)
        
        # AS pointed out by PRVI It works better if the main class is labelled as zero
        labels =  np.array(labels)

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': boxes,
                'labels': labels
            }

        sample = self.transforms(**sample)
        image = sample['image']
        boxes = sample['bboxes']
        labels = sample['labels']
        
        # Normalizing BBOXES
        _,h,w = image.shape
        boxes = A.core.bbox_utils.normalize_bboxes(sample['bboxes'],rows=h,cols=w)
        
        target = {}
        target['boxes'] = torch.as_tensor(boxes,dtype=torch.float32)
        target['labels'] = torch.as_tensor(labels,dtype=torch.long)
        target['image_id'] = torch.tensor([index])

        return image/255, target, image_id

In [15]:
class adv_VinDataset(Dataset):
    def __init__(self, path, transforms=None):
        self.df = pd.read_csv(path + "annotations.csv")
        self.path = path
        self.df["boxes"] = self.df["boxes"].apply(ast.literal_eval)
        self.df["labels"] = self.df["labels"].apply(ast.literal_eval)
        self.image_ids = self.df["image_id"].tolist()
        self.transforms = transforms

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index):
        image_id = self.image_ids[index]
        record = self.df.iloc[index]

        # 讀取圖片
        image = cv2.imread(f'{self.path}/{image_id}.png', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)

        # 讀取標註
        boxes = np.array(record["boxes"], dtype=np.float32)
        labels = np.array(record["labels"], dtype=np.int64)

        # 確保 boxes 是 2D
        if boxes.ndim == 1:
            boxes = np.expand_dims(boxes, axis=0)
            labels = np.expand_dims(labels, axis=0)

        # 確保 labels 為 list
        sample = {
            'image': image,
            'bboxes': boxes.tolist(),
            'labels': labels.tolist()
        }

        # 應用 Albumentations 轉換
        if self.transforms:
            sample = self.transforms(**sample)
            image = sample['image']
            boxes = sample['bboxes']
            labels = sample['labels']

        # 確保 bounding box 格式正確
        boxes = np.array(boxes, dtype=np.float32)
        labels = np.array(labels, dtype=np.int64)

        # 確保 PyTorch 不會出錯
        target = {
            "boxes": torch.as_tensor(boxes, dtype=torch.float32),
            "labels": torch.as_tensor(labels, dtype=torch.long),
            "image_id": torch.tensor([index])
        }

        # 確保 image 是 torch.Tensor 並歸一化
        if not isinstance(image, torch.Tensor):
            image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)

        image = image / 255.0

        return image, target, image_id

## 5. DETR model initialization

In [16]:
import torch.nn.functional as F
class DETRModel(nn.Module):
    def __init__(self,num_classes,num_queries):
        super(DETRModel,self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
        
        for param in self.model.parameters():
            param.requires_grad = True


        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes+1)
        self.model.num_queries = self.num_queries
        
    def forward(self,images):
        return self.model(images)

## 6. Modeling functions

### 6.1 Average meter

In [17]:
# AverageMeter - class for averaging loss,metric,etc over epochs
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

### 6.2 Training Function

Training of DETR is unique and different from FasteRRcnn  and EfficientDET , as we train the criterion as well , the training function can be viewed here : https://github.com/facebookresearch/detr/blob/master/engine.py

In [18]:
def train_fn(data_loader,model,criterion,optimizer,device,scheduler,epoch):
    model.train()
    criterion.train()
    
    summary_loss = AverageMeter()
    
    tk0 = tqdm(data_loader, total=len(data_loader))
    
    check_repeats = []
    for step, (images, targets, image_ids) in enumerate(tk0):
            if image_ids in check_repeats:
                continue
            else:
                check_repeats.append(image_ids)

                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                #print("images : {}".format(images))

                output = model(images)

                loss_dict = criterion(output, targets)
                weight_dict = criterion.weight_dict

                losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

                optimizer.zero_grad()

                losses.backward()
                optimizer.step()
                if scheduler is not None:
                    scheduler.step()

                summary_loss.update(losses.item(),BATCH_SIZE)
                tk0.set_postfix(loss=summary_loss.avg)

    return summary_loss

In [19]:
import torchvision.transforms as T

def show_images(images, adv_images):
    """
    用於顯示原始圖像和對抗性圖像
    """
    transform = T.ToPILImage()  # 將張量轉換為PIL格式
    num_images = min(len(images), 5)  # 限制顯示圖片數量以避免過多
    
    for i in range(num_images):
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        axes[0].imshow(transform(images[i].cpu()))
        axes[0].set_title("Original Image")
        axes[0].axis("off")
        
        axes[1].imshow(transform(adv_images[i].cpu()))
        axes[1].set_title("Adversarial Image")
        axes[1].axis("off")
        
        plt.show()

### 6.3 Evaluation Function

In [20]:
# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    
    # 確保 scale_factors 在 b.device
    scale_factors = torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32, device=b.device)
    
    return b * scale_factors

In [21]:
def eval_fn(data_loader, model,criterion, device):
    model.eval()
    criterion.eval()
    summary_loss = AverageMeter()
    map_df = pd.DataFrame()
    map_df_target = pd.DataFrame()
    
    with torch.no_grad():
        check_repeats_val = []
        tk0 = tqdm(data_loader, total=len(data_loader))
        for step, (images, targets, image_ids) in enumerate(tk0):
            if image_ids in check_repeats_val:
                continue
            else:
                check_repeats_val.append(image_ids)

                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                outputs = model(images)
                outputs['pred_boxes'] = outputs['pred_boxes'].to(device)

                # MAP targets
                for count, label in enumerate(targets[0]['labels']):
                    text = f'{CLASSES[label]}' 
                    xmin = targets[0]['boxes'][count][0] - (targets[0]['boxes'][count][2])/2
                    xmax = targets[0]['boxes'][count][0] + (targets[0]['boxes'][count][2])/2  
                    ymin = targets[0]['boxes'][count][1] - (targets[0]['boxes'][count][3])/2
                    ymax = targets[0]['boxes'][count][1] + (targets[0]['boxes'][count][3])/2

                    data = pd.DataFrame({"ImageID": [image_ids[0]],"LabelName": [text],
                    "XMin": [xmin.item()], "XMax": [xmax.item()], "YMin": [ymin.item()], "YMax": [ymax.item()]})
                    map_df_target = pd.concat([map_df_target, data], ignore_index=True)                

                probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
                keep = probas.max(-1).values > 0.08
                boxes = rescale_bboxes(outputs['pred_boxes'][0, keep], (512,512))
                prob = probas[keep]

                colors = COLORS * 100
                for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):

                    cl = p.argmax()
                    text = f'{CLASSES[cl]}' 
                    
                    # Dataframe for MAP
                    data = pd.DataFrame({"ImageID": [image_ids[0]],"LabelName": [text], "Conf": [p[cl].item()], "XMin": [xmin/512], "XMax": [xmax/512], "YMin": [ymin/512], "YMax": [ymax/512]})
                    map_df = pd.concat([map_df, data], ignore_index=True)     

                loss_dict = criterion(outputs, targets)
                weight_dict = criterion.weight_dict

                losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

                summary_loss.update(losses.item(),BATCH_SIZE)
                tk0.set_postfix(loss=summary_loss.avg)
        
        ann = map_df_target[['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']].values
        det = map_df[['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax']].values
        mean_ap, average_precisions = mean_average_precision_for_boxes(ann, det)

        print("mean_ap : {}".format(mean_ap))
        print("average_precisions : {}".format(average_precisions))
        
    return summary_loss, mean_ap

### 6.4 Iterative train test split

In [22]:
def data_split(train_df, train_ratio, valid_ratio, test_ratio):
    assert train_ratio + valid_ratio + test_ratio == 1, "分配比例之和必須等於 1"
    
    binarizer = MultiLabelBinarizer()
    disease_id = []

    # 構建每個圖像對應的疾病類別列表
    for image_id in train_df.image_id.unique():
        diseases = []
        temp = train_df[train_df["image_id"] == image_id]
        diseases.extend(list(temp["class_id"]))
        disease_id.append(diseases)

    # 將疾病類別進行 one-hot 編碼
    one_hot = binarizer.fit_transform(disease_id)
    
    # 第一步：劃分為訓練集和剩餘部分
    train_size = train_ratio
    remaining_size = 1 - train_size
    
    train_ID, train_label, remaining_ID, remaining_label = iterative_train_test_split(
        np.expand_dims(train_df["image_id"].unique(), axis=1),
        one_hot,
        test_size=remaining_size
    )
    
    # 第二步：劃分剩餘部分為驗證集和測試集
    valid_size = valid_ratio / (valid_ratio + test_ratio)  # 調整比例
    val_ID, val_label, test_ID, test_label = iterative_train_test_split(
        remaining_ID,
        remaining_label,
        test_size=(1 - valid_size)
    )
    
    # 構建訓練、驗證和測試數據集
    training = train_df[train_df["image_id"].isin(train_ID.ravel())]
    validation = train_df[train_df["image_id"].isin(val_ID.ravel())]
    testing = train_df[train_df["image_id"].isin(test_ID.ravel())]
    
    return training, validation, testing

### 6.5 Run DETR

In [23]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [24]:
def run(train_df, fold):
            
    #df_train = train_df[train_df['kfold'] != fold]
    #df_valid = train_df[train_df['kfold'] == fold]
    
    df_train, df_valid, df_test = data_split(train_df, 0.7, 0.1, 0.2)
    
    train_dataset = VinDataset(
    image_ids=df_train.index.values,
    dataframe=df_train,
    transforms=get_train_transforms()
    )

    valid_dataset = VinDataset(
    image_ids=df_valid.index.values,
    dataframe=df_valid,
    transforms=get_valid_transforms()
    )
    
    train_data_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
    )

    valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
    )
    
    # Bipartite Matching Loss
    matcher = HungarianMatcher()
    weight_dict = weight_dict = {'loss_ce': 1, 'loss_bbox': 1 , 'loss_giou': 1}
    losses = ['labels', 'boxes', 'cardinality']

    device = torch.device('cuda')
    model = DETRModel(num_classes=num_classes,num_queries=num_queries)
    
    if torch.cuda.device_count() > 1:
        os.environ["RANK"] = "0"
        os.environ["WORLD_SIZE"] = str(torch.cuda.device_count())
        os.environ["MASTER_ADDR"] = "203.145.216.201" 
        os.environ["MASTER_PORT"] = "52892"
        dist.init_process_group(backend='nccl')
        model = nn.parallel.DistributedDataParallel(model)
        print(f"GPU: {torch.cuda.device_count()}個")

    model = model.to(device)
    
    criterion = SetCriterion(num_classes, matcher, weight_dict, eos_coef = null_class_coef, losses=losses)
    criterion = criterion.to(device)
    
    LR = 5e-5
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
   
    best_loss = 0
    val_loss_track_switch = 0
    all_train_losses = []
    all_valid_losses = []
    all_mean_ap = []
    columns = ['train_losses', 'valid_losses', 'mean_ap']
    df_losses = pd.DataFrame(columns = columns )
    df_losses.to_csv("all_losses.csv",mode='a', index=False)
    for epoch in range(EPOCHS):
        optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
        train_loss = train_fn(train_data_loader, model,criterion, optimizer,device,scheduler=None,epoch=epoch)
        if val_loss_track_switch % 2 == 0: 
            LR = LR/1.12        
            valid_loss, map_validation = eval_fn(valid_data_loader, model,criterion, device)
        val_loss_track_switch = val_loss_track_switch + 1
        
        df_losses = pd.concat([df_losses, pd.DataFrame([{'train_losses': train_loss.avg,'valid_losses': valid_loss.avg,'mean_ap': map_validation}])], ignore_index=True)
        df_losses.to_csv("all_losses.csv",index=False, header=False,mode='a')
        df_losses.drop(df_losses.tail(1).index,inplace=True)
        
        print('|EPOCH {}| TRAIN_LOSS {}| VALID_LOSS {}|'.format(epoch+1,train_loss.avg,valid_loss.avg))
        
        if map_validation > best_loss:
            best_loss = map_validation
            print('Best model found in Epoch {}........Saving Model'.format(epoch+1))
            torch.save(model.state_dict(), f'detr_model.pth')

## 7. Main training function 

In [25]:
n_folds = 5
seed = 42
num_classes = 15
num_queries = 2
null_class_coef = 0.2
BATCH_SIZE = 64
EPOCHS = 16

In [26]:
def model_training():
    train_df = preprocessing()
    import torch, gc
    gc.collect()
    torch.cuda.empty_cache()

    # run this function for training the model
    model = run(train_df, fold=0)
    return

In [27]:
#seed_everything(seed)
#model_training()

## 8. Prediction

### 8.1 Load model

After training the model, comment the line "model = run(fold=0)", and load the model to run the predictions in the test set 

In [28]:
def load_model():
    ## Loading a model
    num_classes = 15
    num_queries = 2
    model = DETRModel(num_classes=num_classes,num_queries=num_queries)
    model.load_state_dict(torch.load("/home/ttban9527/detr_model.pth", map_location=torch.device('cpu')))
    return model

seed_everything(seed)
device = torch.device('cuda')
train_df = preprocessing()
df_train, df_valid, df_test = data_split(train_df, 0.7, 0.1, 0.2)

test_dataset = VinDataset(
    image_ids=df_test.index.values,
    dataframe=df_test,
    transforms=get_valid_transforms()
    )

test_data_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
    )

model = load_model()
model.to(device)

matcher = HungarianMatcher()
weight_dict = weight_dict = {'loss_ce': 1, 'loss_bbox': 1 , 'loss_giou': 1}
losses = ['labels', 'boxes', 'cardinality']

criterion = SetCriterion(num_classes, matcher, weight_dict, eos_coef = null_class_coef, losses=losses)
criterion = criterion.to(device)

test_loss, map_test = eval_fn(test_data_loader, model,criterion, device)

In [29]:
att = "100"

In [30]:
def adv_run():
    
    train_dataset = adv_VinDataset("/home/ttban9527/PGD/detr/train" + att + "/", get_valid_transforms())
    valid_dataset = adv_VinDataset("/home/ttban9527/PGD/detr/valid/", get_valid_transforms())
    
    train_data_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=4,
        collate_fn=collate_fn
    )

    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=4,
        collate_fn=collate_fn
    )
    
    # Bipartite Matching Loss
    matcher = HungarianMatcher()
    weight_dict = weight_dict = {'loss_ce': 1, 'loss_bbox': 1 , 'loss_giou': 1}
    losses = ['labels', 'boxes', 'cardinality']

    device = torch.device('cuda')
    model = DETRModel(num_classes=num_classes,num_queries=num_queries)
    
    if torch.cuda.device_count() > 1:
        os.environ["RANK"] = "0"
        os.environ["WORLD_SIZE"] = str(torch.cuda.device_count())
        os.environ["MASTER_ADDR"] = "203.145.216.201" 
        os.environ["MASTER_PORT"] = "52892"
        dist.init_process_group(backend='nccl')
        model = nn.parallel.DistributedDataParallel(model)
        print(f"GPU: {torch.cuda.device_count()}個")

    model = model.to(device)
    
    criterion = SetCriterion(num_classes, matcher, weight_dict, eos_coef = null_class_coef, losses=losses)
    criterion = criterion.to(device)
    
    LR = 5e-5
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
   
    best_loss = 0
    val_loss_track_switch = 0
    all_train_losses = []
    all_valid_losses = []
    all_mean_ap = []
    columns = ['adv_train_losses', 'adv_valid_losses', 'adv_mean_ap']
    df_losses = pd.DataFrame(columns = columns )
    df_losses.to_csv("adv_all_losses.csv",mode='a', index=False)
    for epoch in range(EPOCHS):
        optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
        train_loss = train_fn(train_data_loader, model,criterion, optimizer,device,scheduler=None,epoch=epoch)
        if val_loss_track_switch % 2 == 0: 
            LR = LR/1.12        
            valid_loss, map_validation = eval_fn(valid_data_loader, model,criterion, device)
        val_loss_track_switch = val_loss_track_switch + 1
        
        df_losses = pd.concat([df_losses, pd.DataFrame([{'adv_train_losses': train_loss.avg,'adv_valid_losses': valid_loss.avg,'adv_mean_ap': map_validation}])], ignore_index=True)
        df_losses.to_csv("adv_all_losses.csv",index=False, header=False,mode='a')
        df_losses.drop(df_losses.tail(1).index,inplace=True)
        
        print('|EPOCH {}| TRAIN_LOSS {}| VALID_LOSS {}|'.format(epoch+1,train_loss.avg,valid_loss.avg))
        
        if map_validation > best_loss:
            best_loss = map_validation
            print('Best model found in Epoch {}........Saving Model'.format(epoch+1))
            torch.save(model.state_dict(), f'adv_detr_' + att+ '.pth')

In [31]:
seed_everything(seed)
import gc
gc.collect()
torch.cuda.empty_cache()
adv_run()

Using cache found in /home/ttban9527/.cache/torch/hub/facebookresearch_detr_main


  0%|          | 0/448 [00:00<?, ?it/s]

  0%|          | 0/4460 [00:00<?, ?it/s]

Number of files in annotations: 1514
Number of files in predictions: 1514
Unique classes: 15
Detections length: 1514
Annotations length: 1514
Aortic enlargement             | 0.131715 |     532
Atelectasis                    | 0.000000 |      35
Calcification                  | 0.000000 |      83
Cardiomegaly                   | 0.059398 |     399
Consolidation                  | 0.000000 |      53
ILD                            | 0.002416 |      94
Infiltration                   | 0.000000 |     129
Lung Opacity                   | 0.001229 |     251
No Finding                     | 0.000000 |    1061
Nodule/Mass                    | 0.000005 |     392
Other lesion                   | 0.000000 |     186
Pleural effusion               | 0.001448 |     235
Pleural thickening             | 0.002825 |     518
Pneumothorax                   | 0.000000 |      14
Pulmonary fibrosis             | 0.001854 |     478
mAP: 0.013393
mean_ap : 0.01339257970608563
average_precisions : {'Aortic enla

  0%|          | 0/448 [00:00<?, ?it/s]

...

  0%|          | 0/448 [00:00<?, ?it/s]

  0%|          | 0/4460 [00:00<?, ?it/s]

Number of files in annotations: 1514
Number of files in predictions: 1505
Unique classes: 15
Detections length: 1505
Annotations length: 1514
Aortic enlargement             | 0.145724 |     532
Atelectasis                    | 0.074410 |      35
Calcification                  | 0.036806 |      83
Cardiomegaly                   | 0.212424 |     399
Consolidation                  | 0.050386 |      53
ILD                            | 0.021566 |      94
Infiltration                   | 0.099175 |     129
Lung Opacity                   | 0.083626 |     251
No Finding                     | 0.000943 |    1061
Nodule/Mass                    | 0.021797 |     392
Other lesion                   | 0.025185 |     186
Pleural effusion               | 0.098587 |     235
Pleural thickening             | 0.021401 |     518
Pneumothorax                   | 0.032499 |      14
Pulmonary fibrosis             | 0.069539 |     478
mAP: 0.066271
mean_ap : 0.06627116939741079
average_precisions : {'Aortic enla

  0%|          | 0/448 [00:00<?, ?it/s]

|EPOCH 16| TRAIN_LOSS 0.5368038948758372| VALID_LOSS 2.5450085093374297|


In [32]:
def adv_load_model():
    ## Loading a model
    num_classes = 15
    num_queries = 2
    model = DETRModel(num_classes=num_classes,num_queries=num_queries)
    model.load_state_dict(torch.load("/home/ttban9527/adv_detr_" + att + ".pth", map_location=torch.device('cpu')))
    return model

In [33]:
seed_everything(seed)
device = torch.device('cuda')
test_dataset = adv_VinDataset("/home/ttban9527/PGD/detr/test/", get_valid_transforms())

test_data_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
    )

model = adv_load_model()
model.to(device)

matcher = HungarianMatcher()
weight_dict = weight_dict = {'loss_ce': 1, 'loss_bbox': 1 , 'loss_giou': 1}
losses = ['labels', 'boxes', 'cardinality']

criterion = SetCriterion(num_classes, matcher, weight_dict, eos_coef = null_class_coef, losses=losses)
criterion = criterion.to(device)

test_loss, map_test = eval_fn(test_data_loader, model,criterion, device)

Using cache found in /home/ttban9527/.cache/torch/hub/facebookresearch_detr_main


  0%|          | 0/130 [00:00<?, ?it/s]

Number of files in annotations: 130
Number of files in predictions: 130
Unique classes: 15
Detections length: 130
Annotations length: 130
Aortic enlargement             | 0.322960 |     123
Atelectasis                    | 0.271709 |       7
Calcification                  | 0.001684 |      27
Cardiomegaly                   | 0.326929 |      91
Consolidation                  | 0.090248 |      35
ILD                            | 0.082793 |      26
Infiltration                   | 0.086809 |      59
Lung Opacity                   | 0.108164 |      91
No Finding                     | 0.000000 |      30
Nodule/Mass                    | 0.048989 |     111
Other lesion                   | 0.010091 |      64
Pleural effusion               | 0.178722 |     103
Pleural thickening             | 0.087241 |     154
Pneumothorax                   | 0.055556 |       3
Pulmonary fibrosis             | 0.076952 |     154
mAP: 0.116590
mean_ap : 0.11658968153166448
average_precisions : {'Aortic enlargem

import csv
import random
import torchvision.transforms as T

# 設置隨機種子和設備
seed_everything(seed)
device = torch.device('cuda')

# 數據預處理
train_df = preprocessing()
df_train, df_valid, df_test = data_split(train_df, 0.7, 0.1, 0.2)

# 加載模型
model = load_model()
model.to(device)

# 準備訓練集和對抗攻擊器
test_dataset = VinDataset(
    image_ids=df_test.index.values,
    dataframe=df_test,
    transforms=get_valid_transforms()
)
attacker = PGDAttacker(5, 10, 2, device=device)

# 設定保存路徑
save_dir = "/home/ttban9527/PGD/detr/test/"
os.makedirs(save_dir, exist_ok=True)
csv_path = os.path.join(save_dir, "annotations.csv")

# 設定攻擊比例
#num_attack = int(len(train_dataset) * 1.0)
#attack_indices = set(random.sample(range(len(train_dataset)), num_attack))

check_repeats = []
count = 0

# 創建 CSV 文件，保存完整的 target 信息
with open(csv_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # 寫入表頭
    csv_writer.writerow(["image_id", "boxes", "labels"])

    # 遍歷數據集
    for idx in tqdm(range(len(test_dataset))):
        # 解析數據集返回的值
        image_tensor, target, image_id = test_dataset[idx]

        image_id = str(image_id)  # 確保 image_id 是字串格式
        boxes = target["boxes"].tolist()  # 邊界框 [N, 4]
        labels = target["labels"].tolist()  # 類別標籤 [N]

        # 將 image_tensor 轉為圖片格式
        to_pil = T.ToPILImage()
        image_tensor = image_tensor.to(device)

        """if idx in attack_indices and image_id not in check_repeats:
            # 進行對抗攻擊
            target_dict = {
                "boxes": target["boxes"].clone().to(device),
                "labels": target["labels"].clone().to(device)
            }
            adv_image_tensor = attacker.attack(image_tensor, target_dict, model)
            adv_image = to_pil(adv_image_tensor.cpu())  # 轉回 PIL 圖片

            new_file_name = f"{image_id}.png"
            count += 1
            if count % 1000 == 0:
                print(f"已經處理了: {count} 張圖片")
        else:
            # 儲存原始圖片"""
        adv_image = to_pil(image_tensor.cpu())  # 轉回 PIL 圖片
        new_file_name = f"{image_id}.png"
        
        # 保存圖片
        new_image_path = os.path.join(save_dir, new_file_name)
        adv_image.save(new_image_path)

        check_repeats.append(image_id)
        

        # 將數據保存到 CSV
        csv_writer.writerow([image_id, boxes, labels])

    print("Total processed: ", count)

print("Image saving and CSV generation complete!")


### 8.2 Plotting results
Plotting expected and predicted boxes with labels


In [34]:
# COCO classes
CLASSES = [
    'Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation',
    'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 
    'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis', 'No Finding'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [35]:
def view_sample_check_MAP(df_valid,model,device, index_to_show):
    map_df = pd.DataFrame()
    map_df_target = pd.DataFrame()    
    
    valid_dataset = VinDataset(
    image_ids=df_valid.index.values,
    dataframe=df_valid,
    transforms=get_valid_transforms()
    )
    
    valid_data_loader = DataLoader(valid_dataset,
                                   batch_size=64,
                                   shuffle=False,
                                   num_workers=4,
                                   collate_fn=collate_fn)
    
    images, targets, image_ids = next(iter(valid_data_loader))
    #print("targets[index_to_show] : {}".format(targets[index_to_show]))
    _,h,w = images[index_to_show].shape # for de normalizing images
    print("h,w  : {}".format(h,w))
    print("targets[index_to_show]['labels']  : {}".format(targets[index_to_show]['labels']))
    images = list(img.to(device) for img in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    annotated_boxes = targets[index_to_show]['boxes'].cpu().numpy()
    print("Anottated boxes.shape AFTER picking ""index_to_show .shape""  : {}".format(annotated_boxes.shape))
    print("Anottated boxes[0] AFTER picking ""index_to_show""  : {}".format(annotated_boxes[0]))
    annotated_boxes = [np.array(box).astype(np.int32) for box in A.core.bbox_utils.denormalize_bboxes(annotated_boxes,h,w)]
    print("denormalize_bboxes Anottated boxes[0] AFTER picking ""index_to_show"" (in coco) : {}".format(annotated_boxes[0]))    
    #annotated_boxes = rescale_bboxes(annotated_boxes[index_to_show], (512,512))     
    
    # MAP targets
    for count, label in enumerate(targets[index_to_show]['labels']):
        print("label : {}".format(label))
        text = f'{CLASSES[label]}' 
        print("text : {}".format(text))
        xmin = targets[index_to_show]['boxes'][count][0] - (targets[index_to_show]['boxes'][count][2])/2
        xmax = targets[index_to_show]['boxes'][count][0] + (targets[index_to_show]['boxes'][count][2])/2  
        ymin = targets[index_to_show]['boxes'][count][1] - (targets[index_to_show]['boxes'][count][3])/2
        ymax = targets[index_to_show]['boxes'][count][1] + (targets[index_to_show]['boxes'][count][3])/2

        data = pd.DataFrame({"ImageID": [image_ids[0]],"LabelName": [text],
        "XMin": [xmin.item()], "XMax": [xmax.item()], "YMin": [ymin.item()], "YMax": [ymax.item()]})
        map_df_target = pd.concat([map_df_target, data], ignore_index=True)
    
    model.eval()
    model.to(device)
    cpu_device = torch.device("cpu")
    
    with torch.no_grad():
        outputs = model(images)   

    # keep only predictions with 0.7+ confidence
    print("outputs['pred_logits'].shape : {}".format(outputs['pred_logits'].shape))
    print("outputs['pred_logits'].softmax(-1).shape : {}".format(outputs['pred_logits'].softmax(-1).shape))
    print("outputs['pred_logits'].softmax(-1)[0, :, :-1].shape : {}".format(outputs['pred_logits'].softmax(-1)[0, :, :-1].shape))
    probas = outputs['pred_logits'].softmax(-1)[index_to_show, :, :-1]
    print("probas.shape : {}".format(probas.shape))
    keep = probas.max(-1).values > 0.5
    print("keep : {}".format(keep))
    # convert boxes from [0; 1] to image scales
    
    print("outputs['pred_boxes'].shape : {}".format(outputs['pred_boxes'].shape))
    #print("outputs['pred_boxes'][index_to_show]: {}".format(outputs['pred_boxes'][index_to_show]))
    
    boxes = rescale_bboxes(outputs['pred_boxes'][index_to_show, keep], (512,512))
    print("Predicted boxes.shape AFTER picking ""index_to_show""  : {}".format(boxes.shape))
    #print("Predicted boxes[0] AFTER picking ""index_to_show"" (in pascal) : {}".format(boxes[0]))
    prob = probas[keep]
    #return probas[keep],bboxes_scaled,image_ids

    string_from_image = f"/home/ttban9527/vinbigdata/train/{image_ids[index_to_show]}.png"
    
    im = Image.open(string_from_image)
    pil_img = im.convert('RGB')
    pil_img.save('colors.jpg')
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    #print("prob : {}".format(prob))
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        print("xmin : {}".format(xmin))
        print("ymin : {}".format(ymin))
        print("xmax : {}".format(xmax))
        print("ymax : {}".format(ymax))
        
        cl = p.argmax()
        print("cl : {}".format(cl))
        text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
        text_df = f'{CLASSES[cl]}'
        
        # Dataframe for MAP
        data = pd.DataFrame({"ImageID": [image_ids[0]],"LabelName": [text_df], "Conf": [p[cl].item()], "XMin": [xmin/512], "XMax": [xmax/512], "YMin": [ymin/512], "YMax": [ymax/512]})
        map_df = pd.concat([map_df, data], ignore_index=True)
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()

# Plot the onces annotated by the doctors
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    #print("prob : {}".format(prob))
    for p, (x0, x1, x2, x3), c in zip(targets[index_to_show]['labels'], annotated_boxes, colors):
        ax.add_patch(plt.Rectangle((x0-x2/2, x1-x3/2), x2, x3,
                                   fill=False, color=c, linewidth=3))
        #print("x0 : {}".format(x0))
        #print("x1 : {}".format(x1))
        #print("x2 : {}".format(x2))
        #print("x3 : {}".format(x3))
        #print("annotated_boxes : {}".format(annotated_boxes))
        cl = p
        print("cl : {}".format(cl))
        text = f'{CLASSES[cl]}'
        ax.text(x0, x1, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()
    
    print("map_df_target : {}".format(map_df_target))
    print("map_df : {}".format(map_df))
    
    ann = map_df_target[['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']].values
    det = map_df[['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax']].values
    mean_ap, average_precisions = mean_average_precision_for_boxes(ann, det)

    print("mean_ap : {}".format(mean_ap))
    print("average_precisions : {}".format(average_precisions))

    return

In [36]:
#train_df = preprocessing()
#model = load_model()
#view_sample_check_MAP(df_test,model=model,device=torch.device('cpu'),index_to_show=10)